<a href="https://colab.research.google.com/github/novello56prod/Produzione-25/blob/main/tempi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

sheet_name = "produzione"
sheet_id = "15-XDrVOSDiTuUnyT_FOZFkZpwZ9k4rsbkTmIHk1-qtE" # The sheet’s ID is the string located between d/ and /edit
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_timbrature = pd.read_csv(url)
qrcodes = df_timbrature["Qrcode"].unique()
#df_timbrature.set_index("Commessa", inplace=True)
df_timbrature = df_timbrature[df_timbrature["Stato"] != "x"]

#df_tempi
columns = ["Qrcode","Nome","Tempo_rilevato"]
df_tempi = pd.DataFrame(columns=columns)
df_tempi.set_index("Qrcode", inplace=True)



for qrcode in qrcodes:
    df_qrcode = df_timbrature[df_timbrature["Qrcode"] == qrcode]
    df_qrcode = df_qrcode.sort_values(by="Data_ora", ascending=True).reset_index(drop=True) # Reset index after sorting
    data_ora_iniziale, data_ora_finale = None, None
    durata_minuti = 0
    inizio = True
    df_tempi.loc[qrcode,"Tempo_rilevato"] = 0
    for i in range(len(df_qrcode)):
        row_qrcode = df_qrcode.loc[i]
        if inizio:
            inizio = False
            data_ora_iniziale = pd.to_datetime(row_qrcode["Data_ora"])
        else:
            data_ora_finale = pd.to_datetime(row_qrcode["Data_ora"])
            durata_minuti = (data_ora_finale - data_ora_iniziale).total_seconds() / 60
            df_tempi.loc[qrcode,"Tempo_rilevato"] = durata_minuti + df_tempi.loc[qrcode,"Tempo_rilevato"]
            inizio = True
            data_ora_iniziale = None
            data_ora_finale = None


df_tempi.reset_index(inplace=True)

from google.colab import data_table
#data_table.enable_dataframe_formatter()
data_table.DataTable(
    df_tempi, include_index=False, num_rows_per_page=30
)

,Qrcode,Nome,Tempo_rilevato
0,NOV25335203TP\n0004,NaN,0
1,NOV25335202TP\n0007,NaN,0
2,NOV25335201TP\n0018,NaN,0
3,NOV25336602TP\n0052,NaN,0
4,NOV25336601TP\n0058,NaN,0
...,...,...,...
621,NOV25325102TF\n0040,NaN,0
622,NOV25337501TP\n0003,NaN,0
623,NOV25337501TF\n0003,NaN,0
624,NOV25336501TF\n0018,NaN,0
